In [3]:
#init spark
!pip install -q pyspark findspark

import findspark
findspark.init()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("analytics_data").getOrCreate()


In [4]:
path = "../data/bank-additional/bank-additional-full.csv"
raw_df = (
    spark.read
    .option("header", True)
    .option("inferschema", True)
    .option("sep", ";")
    .csv(path)
)

cols = ['contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']
target_col = 'y'
df = raw_df.select(*(cols + [target_col]))

df.show(10, truncate=False)

+---------+-----+-----------+--------+--------+-----+--------+---+
|contact  |month|day_of_week|duration|campaign|pdays|previous|y  |
+---------+-----+-----------+--------+--------+-----+--------+---+
|telephone|may  |mon        |261     |1       |999  |0       |no |
|telephone|may  |mon        |149     |1       |999  |0       |no |
|telephone|may  |mon        |226     |1       |999  |0       |no |
|telephone|may  |mon        |151     |1       |999  |0       |no |
|telephone|may  |mon        |307     |1       |999  |0       |no |
|telephone|may  |mon        |198     |1       |999  |0       |no |
|telephone|may  |mon        |139     |1       |999  |0       |no |
|telephone|may  |mon        |217     |1       |999  |0       |no |
|telephone|may  |mon        |380     |1       |999  |0       |no |
|telephone|may  |mon        |50      |1       |999  |0       |no |
+---------+-----+-----------+--------+--------+-----+--------+---+
only showing top 10 rows



In [5]:
print("schema df: ")
df.printSchema()
print("__________________________________________")

print("\n So dong df: ")
print(df.count())
print("__________________________________________")


schema df: 
root
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- y: string (nullable = true)

__________________________________________

 So dong df: 
41188
__________________________________________


In [6]:
from pyspark.sql import functions as F
class_count = df.groupby(target_col).count().orderBy("count", ascending = False)

print("Ty le :")
total = df.count()
func = (F.col("count") / total * 100).cast("double")
class_ratio = class_count.withColumn(
    "ratio (%)", F.round(func, 2)
)
class_ratio.show()

Ty le :
+---+-----+---------+
|  y|count|ratio (%)|
+---+-----+---------+
| no|36548|    88.73|
|yes| 4640|    11.27|
+---+-----+---------+



In [7]:
print("Checking missing values: ")
total = df.count()

for c in df.columns:
    miss = df.filter(F.col(c).isNull()).count()
    print(f"{c:20s} null = {miss}")

Checking missing values: 
contact              null = 0
month                null = 0
day_of_week          null = 0
duration             null = 0
campaign             null = 0
pdays                null = 0
previous             null = 0
y                    null = 0


In [ ]:
from pyspark.sql import functions as F
cat_cols = ['contact', 'month', 'day_of_week', 'y']
for c in cat_cols:
    fre_val = df.groupBy(c).count().orderBy('count', ascending = False)
 
    total = df.count()
    col_expr = ((F.col('count') / total) * 100).cast("double")
    fre_val.withColumn("ratio (%)", F.round(col_expr, 2)).show()


+---------+-----+---------+
|  contact|count|ratio (%)|
+---------+-----+---------+
| cellular|26144|    63.47|
|telephone|15044|    36.53|
+---------+-----+---------+

+-----+-----+---------+
|month|count|ratio (%)|
+-----+-----+---------+
|  may|13769|    33.43|
|  jul| 7174|    17.42|
|  aug| 6178|     15.0|
|  jun| 5318|    12.91|
|  nov| 4101|     9.96|
|  apr| 2632|     6.39|
|  oct|  718|     1.74|
|  sep|  570|     1.38|
|  mar|  546|     1.33|
|  dec|  182|     0.44|
+-----+-----+---------+

+-----------+-----+---------+
|day_of_week|count|ratio (%)|
+-----------+-----+---------+
|        thu| 8623|    20.94|
|        mon| 8514|    20.67|
|        wed| 8134|    19.75|
|        tue| 8090|    19.64|
|        fri| 7827|     19.0|
+-----------+-----+---------+

+---+-----+---------+
|  y|count|ratio (%)|
+---+-----+---------+
| no|36548|    88.73|
|yes| 4640|    11.27|
+---+-----+---------+



### Nhận xét phân tích tần suất các biến phân loại

#### 1. `contact` — Kênh liên hệ
Tỷ lệ **cellular (63.5%)** cao hơn **telephone (36.5%)**.  
Điều này phản ánh đúng thực tế giai đoạn 2008–2010, khi ngân hàng chuyển dần sang gọi qua **di động** để tiếp cận khách hàng nhanh hơn.  

Khi phân tích hiệu quả chiến dịch, cần kiểm tra xem **tỷ lệ “yes”** giữa hai kênh này có khác biệt đáng kể không.  
Nếu di động có tỷ lệ thành công cao hơn, đó là tín hiệu cho việc **ưu tiên kênh liên hệ** này trong các chiến dịch sau.

---

#### 2. `month` — Tháng gọi
Các tháng có nhiều cuộc gọi nhất:
- **May (33%)**
- **July (17%)**
- **August (15%)**
- **June (13%)**

→ Chiếm hơn **75% tổng số cuộc gọi**.  
Có thể đây là **chiến dịch mùa hè**, khi ngân hàng đẩy mạnh huy động vốn hoặc tung sản phẩm mới.

Các tháng khác (Oct, Sep, Mar, Dec) có tỷ lệ rất nhỏ → thường là **off-campaign** hoặc **chiến dịch thử nghiệm**.  
Nên kiểm tra thêm xem **tỷ lệ “yes” có biến động theo mùa** không; ví dụ: tháng May gọi nhiều nhưng hiệu quả có thể không cao.

---

#### 3. `day_of_week` — Ngày gọi
Phân bố **khá đồng đều (khoảng 19–21%)** mỗi ngày.  
Điều này cho thấy ngân hàng triển khai chiến dịch đều trong tuần, **không tập trung riêng vào đầu hoặc cuối tuần**.

Đây là đặc điểm tốt, giúp **loại bỏ bias theo ngày** khi huấn luyện mô hình.  
Tuy nhiên, vẫn nên xem thử **thứ Sáu** có tỷ lệ “yes” cao hơn không — vì khách hàng cuối tuần có thể tâm lý thoải mái hơn.

---

#### 4. `y` — Biến mục tiêu (kết quả)
Dữ liệu **rất mất cân bằng**:  
- **Yes:** 11.3%  
- **No:** 88.7%

Đây là đặc trưng nổi tiếng của **Bank Marketing Dataset**.  
Hệ quả: nếu huấn luyện mô hình mà **không xử lý imbalance**, mô hình sẽ thiên về dự đoán “no”.

→ Ở bước mô hình hóa, cần **cân bằng lớp** bằng:
- `class_weighting`
- `SMOTE`
- hoặc `resampling`

để đảm bảo mô hình học được tín hiệu thực sự của nhóm “yes”.


In [ ]:
#Kiem tra tan suat phan bo cua cac bien so
num_cols = ['duration', 'campaign', 'pdays', 'previous']

df.select(num_cols).describe().show()

percentiles = df.approxQuantile(num_cols, [0.25, 0.5, 0.75, 0.9, 0.95, 0.99], 0.01)
print(percentiles, "\n")


for i, c in enumerate(num_cols):
    res = dict(zip(["25%", "50%", "75%", "90%", "95%", "99%"], percentiles[i]))
    print(f"Percentile cua {c}: ")
    print(res, "\n")

+-------+------------------+-----------------+-----------------+-------------------+
|summary|          duration|         campaign|            pdays|           previous|
+-------+------------------+-----------------+-----------------+-------------------+
|  count|             41188|            41188|            41188|              41188|
|   mean| 258.2850101971448|2.567592502670681|962.4754540157328|0.17296299893172767|
| stddev|259.27924883646455|2.770013542902331|186.9109073447414|0.49490107983928927|
|    min|                 0|                1|                0|                  0|
|    max|              4918|               56|              999|                  7|
+-------+------------------+-----------------+-----------------+-------------------+

[[102.0, 177.0, 312.0, 526.0, 702.0, 4918.0], [1.0, 2.0, 3.0, 5.0, 6.0, 56.0], [999.0, 999.0, 999.0, 999.0, 999.0, 999.0], [0.0, 0.0, 0.0, 1.0, 1.0, 7.0]] 

Percentile cua duration: 
{'25%': 102.0, '50%': 177.0, '75%': 312.0, '90%': 5

### Nhận xét phân tích các biến số

#### 1. `duration` — Thời lượng cuộc gọi
- Trung bình khoảng **258 giây**, độ lệch chuẩn gần bằng trung bình → **phân bố rất lệch phải (right-skewed)**.  
- **75%** cuộc gọi kéo dài ≤ **312 giây**, nhưng **1% trên cùng** lên tới gần **5000 giây (~82 phút)** → có **outlier mạnh**.  
- Điều này hợp lý, vì đa số khách hàng từ chối sớm; còn các cuộc gọi dài thường đến từ khách hàng có hứng thú hoặc nhân viên thuyết phục lâu.  
- Tuy nhiên, như đã nói ở phần trước, `duration` **không nên dùng để dự đoán trực tiếp**, vì đây là **kết quả sau khi gọi**, không phải thông tin biết trước.  
  → Dùng trong **EDA (phân tích khám phá dữ liệu)** để hiểu hành vi khách hàng là hợp lý.

---

#### 2. `campaign` — Số lần liên hệ trong chiến dịch hiện tại
- **Trung vị = 2**, **75% ≤ 3**, nhưng **1% cao nhất** lên tới **56 lần** → có những khách hàng bị gọi **hơn 50 lần (!)**  
- Đây là **outlier rõ ràng**, nhưng lại chứa **thông tin hành vi**: chiến dịch đã cố gắng “đuổi theo” khách hàng đó.  
- Khi mô hình hóa, nên:
  - **Giới hạn (clip)** giá trị tối đa ở một ngưỡng hợp lý (ví dụ 10), hoặc  
  - **Log-transform** để giảm ảnh hưởng của các giá trị cực lớn.

---

#### 3. `pdays` — Số ngày từ lần liên hệ trước
- Tất cả các percentile đều = **999**, cho thấy **phần lớn khách hàng chưa từng được liên hệ trước đây**.  
- Theo mô tả dữ liệu, **999 nghĩa là “no previous contact”**.  
- Biến này ít thông tin trực tiếp, nên khi xử lý có thể:
  - Tạo biến nhị phân mới: `has_contact_before = (pdays != 999)`, hoặc  
  - Giữ nguyên 999 và để mô hình tự học (tùy thuật toán).

---

#### 4. `previous` — Số lần liên hệ trước chiến dịch hiện tại
- **75% khách hàng chưa từng được liên hệ (0)**, **95% ≤ 1**, chỉ vài người đến **7 lần**.  
- Nghĩa là **đa số là khách hàng mới**.  
- Biến này có **phân bố rất lệch (one-sided)**, nên khi đưa vào mô hình cần:
  - **Chuẩn hóa (scaling)** hoặc  
  - **Binning (phân nhóm)** để mô hình dễ học và ổn định hơn.


### 1. Insight về hành vi liên hệ (kênh – thời điểm – kết quả):
→ Mục tiêu: tìm hiểu kênh và thời điểm nào hiệu quả nhất.

#### 2: “Tần suất & lịch sử liên hệ.”

In [46]:
contacted_before_df = df.filter(df['pdays'] != 999)
contacted_before_df.show(5)

print("--------------------------------------")
has_contact_bf_nums = contacted_before_df.count()
total = df.count()
ratio_of_contact = round(has_contact_bf_nums / total * 100, 2)

print("Tỷ lệ khách hàng đã từng được gọi trước đó (không phải lần đầu liên hệ):")
print(f"{ratio_of_contact:.2f}% trên tổng số {total} khách hàng.")


+---------+-----+-----------+--------+--------+-----+--------+---+
|  contact|month|day_of_week|duration|campaign|pdays|previous|  y|
+---------+-----+-----------+--------+--------+-----+--------+---+
|telephone|  nov|        wed|     119|       1|    6|       1| no|
| cellular|  nov|        mon|     112|       1|    4|       1| no|
| cellular|  nov|        mon|      94|       1|    4|       1| no|
| cellular|  nov|        mon|      77|       1|    3|       1| no|
| cellular|  nov|        mon|     200|       2|    4|       1| no|
+---------+-----+-----------+--------+--------+-----+--------+---+
only showing top 5 rows

--------------------------------------
Tỷ lệ khách hàng đã từng được gọi trước đó (không phải lần đầu liên hệ):
3.68% trên tổng số 41188 khách hàng.


### Nhận xét ngắn gọn

Chỉ **3.68%** khách hàng từng được gọi trước đó, nghĩa là **hơn 96%** là lần đầu tiên được liên hệ.  

Điều này xác nhận chiến dịch marketing của ngân hàng chủ yếu là **cold call**, không phải **chăm sóc khách hàng cũ**.  

Nhóm khách hàng “được gọi lại” tuy nhỏ, nhưng **có thể là nhóm tiềm năng cao hơn** — nên ở phần sau, bạn nên kiểm tra xem họ có **tỷ lệ “yes” cao hơn** hay không.  
Đó là một **insight quan trọng** để mô hình hóa hành vi **follow-up** trong chiến dịch.
